# Summary


----

# Imports

In [27]:
%run imports.ipynb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2016-08-25 22:46:19.583880


In [32]:
%run db.ipynb

MySQL database already running...


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2016-08-25 22:46:27.094379


In [33]:
NOTEBOOK_NAME = 'cosmic'
os.makedirs(NOTEBOOK_NAME, exist_ok=True)

# Export from remote db

In [56]:
DB_URL = kmtools.db_tools.parse_connection_string(db_remote.connection_string)['db_url']

In [57]:
# SQL query
sql_query = r"""\
SELECT 
    us.uniprot_acc,
    v.mutationassessor_variant mutation,
    NULL ddg_exp,
    cme.fathmm_score del_score_exp,
    CASE cme.fathmm_prediction WHEN 'NEUTRAL' THEN 0 WHEN 'PATHOGENIC' THEN 1 ELSE NULL END del_class_exp
INTO OUTFILE '/tmp/{}.tsv' 
    FIELDS TERMINATED BY '\t' OPTIONALLY ENCLOSED BY '"'
    LINES TERMINATED BY '\n'
FROM cosmic_new.cosmic_coding_muts ccm 
JOIN cosmic_new.cosmic_mutant_export cme ON (cme.mutation_id = ccm.id)
JOIN dbnsfp.variant v ON (ccm.chrom = v.chr AND ccm.pos = v.pos_1based AND ccm.ref = v.ref AND ccm.alt = v.alt)
JOIN uniprot_kb_proteomes.UP000005640_9606_fasta us ON (v.mutationassessor_uniprotid = us.uniprot_id)
WHERE cme.fathmm_score IS NOT NULL AND cme.fathmm_prediction IS NOT NULL;
""".format(NOTEBOOK_NAME)
print_sql(sql_query)

In [58]:
db_remote.engine.execute(sql_query)

# Load to local db

In [59]:
!rsync -av {DB_URL}:/tmp/{NOTEBOOK_NAME}.tsv {NOTEBOOK_NAME}/{NOTEBOOK_NAME}.tsv

receiving incremental file list
cosmic.tsv

sent 60,026 bytes  received 61,104,164 bytes  3,306,172.43 bytes/sec
total size is 61,089,147  speedup is 1.00


In [60]:
!git lfs track {NOTEBOOK_NAME}/{NOTEBOOK_NAME}.tsv

cosmic/cosmic.tsv already supported


In [61]:
!git add {NOTEBOOK_NAME}/{NOTEBOOK_NAME}.tsv -f

In [84]:
t = db.import_file(
    op.join(NOTEBOOK_NAME, NOTEBOOK_NAME + '.tsv'),
    tablename=NOTEBOOK_NAME + '_tmp',
    names=['uniprot_id', 'uniprot_mutation', 'ddg_exp', 'del_score_exp', 'del_class_exp'],
    sep='\t',
    na_values=['\\N'],
    keep_tmp=True,
)

In [85]:
t.name

'cosmic_tmp'

In [86]:
db.engine.execute("DROP TABLE IF EXISTS {}".format(NOTEBOOK_NAME))

In [87]:
db.engine.execute("""\
CREATE TABLE {} AS (
    SELECT 
    uniprot_id, 
    uniprot_mutation, 
    NULL ddg_exp, 
    AVG(del_score_exp) del_score_exp, 
    ROUND(AVG(del_class_exp)) del_class_exp
    FROM {}
    GROUP BY uniprot_id, uniprot_mutation
)
""".format(NOTEBOOK_NAME, NOTEBOOK_NAME + '_tmp'))

In [88]:
db.engine.execute("DROP TABLE IF EXISTS {}".format(NOTEBOOK_NAME + '_tmp'))

In [89]:
t.name = t.name.replace('_tmp', '')

In [90]:
t.add_idx_column()

991617

In [91]:
t.create_indexes([
        (['uniprot_id', 'uniprot_mutation'], True),
    ])

In [92]:
t.compress()

File size before: 37.84 MB
File size after: 18.06 MB
File size savings: 18.06 MB (47.73 %)


# Git LFS

In [93]:
!git lfs track ./mysqld/{os.environ['DB_SCHEMA']}/{NOTEBOOK_NAME}.*

Tracking ./mysqld/mutation_sets/cosmic.MYD
Tracking ./mysqld/mutation_sets/cosmic.MYI
Tracking ./mysqld/mutation_sets/cosmic.frm


In [94]:
!git add -f ./mysqld/{os.environ['DB_SCHEMA']}/{NOTEBOOK_NAME}.*